In [13]:
# Package imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
list_of_files = {}
df = pd.DataFrame()
for (dirpath, dirnames, filenames) in os.walk('data/'):
    for filename in filenames:
        if filename.endswith('.txt'): 
            temp_df = pd.read_csv(os.sep.join([dirpath, filename]), '\t')
            row_dict = {'cancer': 'path'}
            for index, row in temp_df.iterrows():
                #row_id = temp_df.loc[row,'miRNA_ID']
                row_id = temp_df.iloc[index,0]
                row_dict[row_id] = temp_df.iloc[index,2]
                #row_dict[row_id] = temp_df.loc[row,'reads_per_million_miRNA_mapped']
            df = df.append(row_dict, ignore_index = True)

In [15]:
print(list_of_files)

#each file corresponds to a row we want in pandas dataframe

{'MANIFEST.txt': 'data/Uveal Melanoma/MANIFEST.txt', '02ac46fc-c8fe-4604-bb3c-68b3843ebd0e.mirbase21.mirnas.quantification.txt': 'data/Breast Invasive Carcinoma/00100f1e-f293-481d-8189-285eaf73dfc2/02ac46fc-c8fe-4604-bb3c-68b3843ebd0e.mirbase21.mirnas.quantification.txt', 'dece924e-2e46-497c-8405-52b40e072450.mirbase21.mirnas.quantification.txt': 'data/Breast Invasive Carcinoma/004ca2d1-af92-45fd-aef9-f7a84131f467/dece924e-2e46-497c-8405-52b40e072450.mirbase21.mirnas.quantification.txt', '7ce26f09-3fa4-4b2e-b7b0-716ef2fd42b4.mirbase21.mirnas.quantification.txt': 'data/Breast Invasive Carcinoma/00eaa502-82ee-46c6-838f-656c68d8e336/7ce26f09-3fa4-4b2e-b7b0-716ef2fd42b4.mirbase21.mirnas.quantification.txt', '17f0325a-dba6-438a-9a6b-07b64c223f42.mirbase21.mirnas.quantification.txt': 'data/Breast Invasive Carcinoma/00ff4750-d1a4-442d-bc99-505e278cdc3d/17f0325a-dba6-438a-9a6b-07b64c223f42.mirbase21.mirnas.quantification.txt', 'annotations.txt': 'data/Pancreatic Adenocarcinoma/ffdd0bba-1a6d-45